In [2]:
from utils.generation import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

# download and load all models
preload_models(r'exp\valle_dev\epoch-2.pt')

# generate audio from text
text_prompt = """
我今天想吃牛肉麵
"""
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("vallex_generation.wav", SAMPLE_RATE, audio_array)

# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BBS\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\BBS\AppData\Local\Temp\jieba.cache
Loading model cost 0.305 seconds.
DEBUG:jieba:Loading model cost 0.305 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


VALL-E EOS [0 -> 186]
